# Chapter 21: Asynchronous Programming

> The problem with normal approaches to asynchronous programming is that they're all-or-nothing propositions. You rewrite all your code so none of it blocks or you're just wasting your time. 
> -- Alvaro Videla and Jason J. W. Williams, RabbitMQ in Action

This chapter addresses three major topics that are closely related:

- Python's `async def`, `await`, `async with`, and `async for` constructs
- Objects supporting those constructs: native coroutines and asynchronous variants of context managers, iterables, generators, and comprehensions
- `asyncio` and other asynchronous libraries

## A Few Definitions

- Native coroutine: A coroutine function defined with `async def`. You can delegate from a native coroutine to another native coroutine using the `await` keyword, similar to how classic coroutines use `yield from`. The `async def` statement always defines a native coroutine, even if the `await` keyword is not used in its body. The `await` keyword cannot be used outside of a native coroutine.

- Classic coroutine: A generator function that consumes data sent to it via `my_coro.send(data)` calls, and reads that data by using `yield` in an expression. Classic coroutines can delegate to other classic coroutines using `yield from`. Classic coroutine cannot be driven by `await`, and are no longer supported by `asyncio`.

- Generator-based coroutine: A generator function decorated with `@types.coroutine` -- introduced in Python 3.5. That decorator makes the generator compatible with the new `await` keyword. It's in the process of being deprecated in favor of native coroutines.

- Asynchronous generator: A generator function defined with `async def` and using `yield` in its body. It returns an asynchronous generator object that provides `__anext__`, a coroutine method to retrieve the next item.

## An asyncio Example: Probing Domains

### Example 21-1. blogdom.py: search for domains for a Python blog

In [ ]:
import asyncio
import socket
from keyword import kwlist

MAX_KEYWORD_LEN = 4 #1

async def probe(domain: str) -> tuple[str, bool]: #2
    loop = asyncio.get_running_loop() #3
    try:
        await loop.getaddrinfo(domain, None) #4
    except socket.gaierror:
        return domain, False
    return domain, True

async def main(): #5
    names = (kw for kw in kwlist if len(kw) <= MAX_KEYWORD_LEN) #6
    domains = (f'{name}.dev'.lower() for name in names) #7
    coros = [probe(domain) for domain in domains] #8
    for coro in asyncio.as_completed(coros): #9
        domain, found = await coro  #10
        mark = '+' if found else '-' 
        print(f'{mark} {domain}')
        
if __name__ == '__main__':
    asyncio.run(main()) #11


1. Set maximum length of keyword for domains, because shorter is better.

2. `probe` returns a tuple with the domain name and a boolean; `True` means the domain resolved. Return the domain name will make it easier to display the results.

3. Get a reference to the `asyncio` event loop, so we can use it next.

4. The `loop.getaddrinfo()` coroutine-method returns a five-part tuple of parameters to connect to the given address using a socket. In this example, we don't need the result. If we got it, the domain resolves; otherwise, it doesn't.

5. `main` must be a coroutine, so that we can use `await` in it.

6. Generator to yield Python keywords with length to `MAX_KEYWORD_LEN`.

7. Generator to yield domains with the `.dev` suffix.

8. Build a list of coroutine objects by invoking the `probe` coroutine with each `domain` argument.

9. `asyncio.as_completed` is generator that yields coroutines that return the results of the coroutines passed to it in the order they are completed--not the order they were submitted. It's similar to `future.as_completed` in the previous chapter.

10. At this point, we know the coroutines is done because that's how `as_completed` works. Therefore, the `await` expression will not block but we need it to get the result from `coro`. If `coro` raised an unhandled exception, it would be re-raised here.

11. `asyncio.run` starts the event loop and return only when the event loop exits. This is a common pattern for scripts that use `asyncio`: implement `main` as a coroutine, and drive it with `asyncio.run` inside the `if __name__ == __main__:` block.

### Guido's Trick to Read Async Code

A trick suggested by Guido van Rossum himself: squint and pretend `async` and `await` keywords are not there. If you do that, you'll realize that coroutines read like plain old sequential functions.

## New Concept: Awaitable

The `await` keyword works with `awaitables`. As an end user of `asyncio`, these are the awaitables you'll encounter:

-  A `native coroutine object`, which you get by calling a `native coroutine function`.

- An `asyncio.Task`, which you usually get by passing a coroutine object to `asyncio.create_task()`.

## Downloading with asyncio and HTTPX

### Example 21-3. flags_asyncio.py: imports and download functions

In [3]:
import asyncio

from httpx import AsyncClient #1
from flags import BASE_URL, save_flag, main #2

async def download_one(client: AsyncClient, cc: str): #3
    image = await get_flag(client, cc) 
    save_flag(image, f'{cc}.gif')
    print(cc, end=' ', flush=True)
    return cc


async def get_flag(client: AsyncClient, cc: str) -> bytes: #4
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower() 
    resp = await client.get(url, timeout=6.1,
                            follow_redirects=True) #5
    return resp.read() #6

1. `httpx` must be installed--it's not part of the standard library.

2. Reuse code from `flags.py`.

3. `download_one` must be a native coroutine, so it can `await` on `get_flag`--which does the `HTTP` request. Then it displays the code of the downloaded flag, and saves the image.

4. `get_flag` needs to receive the `AsyncClient` to make the request.

5. The `get` method of an `httpx.AsyncClient` instance returns a `ClientResponse` object that is also an asynchronous context manager.

6. Network I/O operations are implemented as coroutine methods, so they are driven by the `asyncio` event loop.

### Example 21-2. flags_asynico.py: startup functions

In [ ]:
def download_many(cc_list: list[str]) -> int: #1
    return asyncio.run(supervisor(cc_list)) #2

async def supervisor(cc_list: list[str]) -> int: 
    async with AsyncClient() as client: #3
        to_do = [download_one(client, cc) 
                    for cc in sorted(cc_list)] #4
        res = await asyncio.gather(*to_do) #5
    return len(res) #6
    
if __name__ == '__main__':
    main(download_many) 

1. This needs to be a plain function--not a coroutine--so it can be passed to and called by the `main` function from the `main` function.

2. Execute the event loop driving the `supervisor(cc_list)` coroutine object until it returns. This will block while the event loop runs. The result of this line is whatever the `supervisor` coroutine returns.

3. `Asynchronous` HTTP client operations in `httpx` are methods of `AsyncClient`, which is also an asynchronous context manager: a context manager with asynchronous setup and teardown methods.

4. Build a list of coroutine objects by calling the `download_one` coroutine once for each flag to be retrieved.

5. Wait for the `asyncio.gather` coroutine, which accepts one or more awaitable arguments and waits for them to complete, returning a list of results for the given awaitables in the order they were submitted.

6. `supervisor` returns the length of the list returned by `asyncio.gather`.

asyncio.as_completed方法和asyncio.gather方法是Python asyncio库中用于并发执行协程的两种常用方法。它们在处理并发任务时有一些区别和特点。

1. asyncio.as_completed方法：
   - asyncio.as_completed方法接受一个协程列表作为参数，并返回一个可迭代对象。
   - 通过使用await关键字，可以逐个获取已完成的协程的结果。
   - 当有协程完成时，as_completed方法会立即返回该协程的结果，而不需要等待其他协程完成。
   - 这种方法适用于需要按照完成顺序逐个处理结果的情况。

2. asyncio.gather方法：
   - asyncio.gather方法接受一个或多个协程作为参数，并返回一个新的协程。
   - 通过使用await关键字，可以获取所有协程的结果。
   - gather方法会等待所有协程都完成后才返回结果，即所有协程都执行完毕后才会继续执行后续代码。
   - 这种方法适用于需要同时处理多个协程结果的情况。

总结：
- asyncio.as_completed方法适用于需要按照完成顺序逐个处理结果的情况，而asyncio.gather方法适用于需要同时处理多个协程结果的情况。
- asyncio.as_completed方法可以在协程完成时立即获取结果，而asyncio.gather方法需要等待所有协程完成后才返回结果。
- 根据具体需求选择合适的方法可以更好地处理并发任务。

### The Secret of Native Coroutine: Humble Generators

![asyncio event loop](./img/2023-11-24-14-33-22.png)

A key difference between the classic coroutine and native coroutine is that there are no visible `.send()` calls or `yield` expressions in the latter. This is illustrated in Figure 21-1.

Under the hood, the `asyncio` event loop makes the `.send()` calls that drive your coroutines, and your coroutines `await` on other coroutines, including library coroutines. As mentioned, `await` borrows most of implementation from `yield from`, which also makes `.send` calls to drive coroutines.

The `await` chain eventually reaches a low-level awaitable, which returns a generator that the event loop can drive in response to events such as timers or network I/O. The low-level awaitables and generators at the end of these `await` chains are implemented into the libraries, are not part of their APIs, and maybe Python/C extensions.

Using functions like `asyncio.gather` and `asyncio.created_task`, you can start multiple concurrent `await` channels, enabling concurrent execution of multiple I/O operations driven by a single event loop, in a single thread.

### The All-or-Nothing Problem

For peak performance with `asyncio`, we must replace every function that does I/O with an asynchronous with `asyncio`, we must replace every function that does I/O with an asynchronous version that is activated with `await` or `asyncio.create_task`, so that control is given back to the event loop while the function waits for I/O. If you can't rewrite a blocking function as a coroutine, you should run it in a separate thread or process.


How can you run a thread or process in a coroutine?

Short answer: use `ThreadPoolExecutor` and `as_completed` in `concurrent.futures` module.

## Asynchronous Context Managers

### Example 21-4. Sample code from documentation of the asyncpg PostgreSQL driver

```python
tr = connection.transaction()

await tr.start()
try:
    await connection.execute()
except:
    await tr.rollback()
    raise
else:
    await tr.commit()
```

A database transaction is a natural fit for the context manager protocol: the transaction has to be started, data is changed with `connection.execute`, and then a rollback or commit must happen, depending on the outcome of the operation.

In an asynchronous driver like `asyncpg`, the setup and wrap-up need to be coroutines so that other operations can happen concurrently. However, the implementation of the classic `with` statement doesn't support coroutines doing the work of `__enter__` and `__exit__`. 

That's why PEP 492--Coroutines with async and await syntax introduced the `async with` statement, which works with asynchronous context manager: objects implementing the `__aenter__` and `__aexit__` methods as coroutines.

With `async with`, Example 21-4 can be written like this other snippet from the `asyncpg` documentation:

```python
async with connection.transaction():
    await connection.execute("INSERT INTO mytable VALUES (1, 2, 3)")
```

In the `asyncpg.Transaction class`, the `__aenter__` coroutine method does `await self.start()`, and `__aexit__` does `await self.commit()` or `await self.rollback()` depending on whether an exception was raised in the `async with` block. Using coroutines to implement `Transaction` as an asynchronous context manager allows `asyncpg` to handle many transactions concurrently.

## Enhancing the asyncio downloader

### Using asyncio.as_completed and a Thread

### Example 21-6. flags2_asyncio.py script

In [2]:
import asyncio
from collections import Counter
from http import HTTPStatus
from pathlib import Path

import httpx
import tqdm

from flags2_common import main, save_flag, DownloadStatus

# low concurrency default to avoid errors from remote site,
# such as 503 - Service Temporarily Unavailable

DEFAULT_CONCUR_REQ = 5 
MAX_CONCUR_REQ = 1000 

async def get_flag(client: httpx.AsyncClient, #1
                    cc: str,
                    base_url: str) -> bytes:
    url = f'{base_url}/{cc}/{cc}.gif'.lower()
    resp = await client.get(url, timeout=6.1,
                            follow_redirects=True) #2
    resp.raise_for_status()
    return resp.read()

async def download_one(client: httpx.AsyncClient,
                        cc: str,
                        base_url: str,
                        semahore: asyncio.Semaphore,
                        verbose: bool) -> DownloadStatus:
    try:
        async with semahore: #3
            image = await get_flag(client, cc, base_url)
    except httpx.HTTPStatusError as exc: #4
        res = exc.response
        if res.status_code != 404:
            raise
        status = DownloadStatus.not_found
        msg = f'not found: {res.url}'
    else:
        await asyncio.to_thread(save_flag, image, f'{cc}.gif') #5
        status = DownloadStatus.ok
        msg = 'OK'
    if verbose and msg:
        print(cc, msg)
    return status

1. `get_flag` is very similar to the sequential version in Example 20-14. First difference: it requires the `client` parameter.

2. Second and third differences: `.get` is an `AsyncClient` method, and it's a coroutine, so we need to await it.

3. Use the `semaphore` as an asynchronous context manager so that the program as a whole is not blocked; only this coroutine is suspended when the semaphore counter is zero. 

4. The error handling logic is the same as in `download_one`.

5. Saving the image is an I/O operation. To avoid blocking the event loop, run `save_flag` in a thread.

### Throttling Requests with a Semaphore

Network clients like the ones we are studying should be throttled(i.e. limited) to avoid pounding the server with too many concurrent requests.

A semaphore is a synchronization primitive, more flexible that a lock. A semaphore can be held by multiple coroutines, with a configurable maximum number. This makes it ideal to throttle the number of active concurrent coroutines.

In `flags2_threadpool.py` Example 20-16, the throttling was done by instantiating the `ThreadPoolExecutor` with the required `max_workers` argument set to `concur_req` in the `download_many` function. In `flag2_asyncio.py`, an `asyncio.Semaphore` is created by the supervisor function and passed as the `semaphore` argument to `download_one`.

### Python's Semaphores

Computer scientist Edsger W.Dijkstra invented the `semaphore` in the early 1960s. It's a simple idea, but it's so flexible that most other synchronization objects--such as locks and barriers--can be built on top of semaphores. There are three Semaphore classes in Python's standard library: one in threading, another in multiprocessing, and a third one in `asyncio`. Here we'll describe the latter.

An `asyncio.Semaphore` has an internal counter that is decremented whenever we `await` on the `.acquire()` coroutine method, and incremented when we call the `.release()` method--which is not a coroutine because it never blocks. The initial value of the counter is set when the `Semaphore` is instantiated:

```python
semaphore = asyncio.Semaphore(concur_req)
```

Awaiting on `.acquire()` causes no delay when the counter is greater than zero, but if the counter is zero, `.acquire()` suspends the awaiting coroutine until some other coroutine calls `.release()` on the same `Semaphore`, thus incrementing the counter. Instead of using those methods directly, it's safer to use the `semaphore` as an asynchronous context manager:

```python
async with semaphore:
    image = await get_flag(client, base_url, cc)
```

The `Semaphore.__aenter__` coroutine method await for `.acquire()`, and its `__aexit__` coroutine method calls `.release()`. That snippet guarantees that no more than `concur_req` instances of `get_flags` coroutines will be active at any time.

Each of the `Semaphore` classes in the standard library has a `BoundedSemaphore` subclass that enforces an additional constraints: the internal counter can never become larger than the initial value when there are more `.release()` than `.acquire()` operations.


### Example 21-7. flags2_asyncio.py: script continued from Example 21-6

In [ ]:
async def supervisor(cc_list: list[str],
                        base_url: str,
                        verbose: bool,
                        concur_seq: int) -> Counter[DownloadStatus]: #1
    counter: Counter[DownloadStatus] =  Counter()
    semaphore = asyncio.Semaphore(concur_seq) #2
    async with httpx.AsyncClient() as client:
        to_do = [download_one(client, cc, base_url, semaphore, verbose)
                    for cc in sorted(cc_list)] #3
        to_do_iter = asyncio.as_completed(to_do) #4
        if not verbose:
            to_do_iter = tqdm.tqdm(to_do_iter, total=len(cc_list)) #5
        error: httpx.HTTPError | None = None #6
        for coro in to_do_iter: #7
            try:
                status = await coro #8
            except httpx.HTTPStatusError as exc:
                error_msg = 'HTTP error {resp.status_code} - {resp.reason_phrase}'
                error_msg = error_msg.format(resp=exc.response)
                error = exc #9
            except httpx.RequestError as exc:
                error_msg = f'{exc} {type(exc)}'.strip()
                error = exc #10
            except KeyboardInterrupt:
                break
            
            if error:
                status = DownloadStatus.ERROR #11
                if verbose:
                    url = str(error.request.url) #12
                    cc = Path(url).stem.upper() #13
                    print(f'{cc} error: {error_msg}')
            counter[status] += 1
    return counter


def download_many(cc_list: list[str],
                    base_url: str,
                    verbose: bool,
                    concur_req: int) -> Counter[DownloadStatus]:
    coro = supervisor(cc_list, base_url, verbose, concur_req)
    return asyncio.run(coro) #14
                

if __name__ == '__main__':
    main(download_many, DEFAULT_CONCUR_REQ, MAX_CONCUR_REQ)        
    

1. `supervisor` takes the same arguments as the `download_many` function, but it can not be invoked directly from `main` because it's a coroutine and not a plain function like `download_many`.

2. Create an `asyncio.Semaphore` that will be used to throttle the number of concurrent downloads. The value of `concur_req` is computed by the `main` function based on command-line arguments and constants set in each example.

3. Create a list of coroutine objects, one per call to the `download_one` coroutine.

4. Get an iterator that will return coroutine objects as they are done. I did not place this call to `as_completed` directly in the `for` loop because I may need to warp it with the `tqdm` iterator for the progress bar, depending on the user's choice for verbosity.

5. Wrap the `as_completed` iterator with the `tqdm` generator function to display progress.

6. Declare and initialize `error` with `None`; this variable will be used to hold an exception beyond the `try/except` statement, if one is raised.

7. Iterate over the completed coroutine objects; this loop is similar to the one in `download_many` in Example 20-16.

8. `await` on the coroutine to get its result. This will not block because `as_completed` only produces coroutines that are done.

9. This assignment is necessary because the `exc` variable scope is limited to this except clause, but I need to preserve its value for latter.

10. Same as before.

11. If there was an error, set the `status`.

12. In verbose mode, extract the URL from the exception that was raised...

13. ...and extract the name of the file to display the country code next.

14. `download_many` instantiates the `supervisor` coroutine object and passes it to the event loop with `asyncio.run`, collecting the counter `supervisor` returns when the event loop ends.

### Making Multiple Requests for Each Download

In [ ]:
# Example 21-8. flags_asyncio.py: get_country coroutine

async def get_country(client: httpx.AsyncClient,
                    base_url: str,
                    cc: str) -> bytes: #1
    url = f'{base_url}/{cc}/metadata.json'.lower()
    resp = await client.get(url, timeout=3.1,
                            follow_redirects=True)
    resp.raise_for_status()
    metadata = resp.json() #2
    return metadata['country'] #3

1. This coroutine returns a string with the country name--if all goes well.

2. `metadata` will get a Python `dict` built from the JSON contents of the response.

3. Return the country name.

### Example 21-9. flags_asyncio.py: download_one coroutine

In [ ]:
async def download_one(client: httpx.AsyncClient,
                        cc: str,
                        base_url: str,
                        semaphore: asyncio.Semaphore,
                        verbose: bool) -> DownloadStatus:
    try:        
        async with semaphore:
            image = await get_flag(client, cc, base_url) #1
        async with semaphore:
            country = await get_country(client, base_url, cc) #2
    except httpx.HTTPStatusError as exc:
        res = exc.response
        if res.status_code == 404:
            status = DownloadStatus.NOT_FOUND
            msg = f'not found: {res.url}'
        else:
            raise
    else:
        filename = country.replace(' ', '_') #3
        await asyncio.to_thread(save_flag, image, f'{filename}.gif')
        status = DownloadStatus.OK
        msg = 'OK'
    if verbose and msg:
        print(cc, msg)
    return status 
    

1. Hold the `semaphore` to `await` for `get_flag`...

2. ...and again for `get_country`

3. Use the country name to create a filename. As a command-line user, I don't to see spaces in filenames.

### Why use two semaphores here? How about one?

- Put the calls to `get_flag` and `get_country` in separate `with` blocks controlled by the `semaphore` because it's good practice to hold semaphores and locks for the shortest possible time.

- Put the calls to `get_flag` and `get_country` in one `with` block can cause RuntimeError: 'RuntimeError: Cannot send a request, as the client has been closed.'

- The second `with` block will execute after the first `with` block finish all tasks.

One challenge is to know when you have to use `await` and when you can't use it. The answer in principle is easy: you `await` coroutines and other awaitables, such as `asyncio.Task` instances. But some APIs are tricky, mixing coroutines and plain functions in seemingly arbitrary ways, like the `StreamWriter` class we'll use in Example-14.

## Delegating Tasks to Executors

In Python 3.9, you can use `asyncio.to_thread`:

```python
await asyncio.to_thread(save_flag, image, filename)
```

If you need to support Python 3.7 and 3.8, you can use `run_in_executor`:

```python
loop = asyncio.get_event_loop() #1
loop.run_in_executor(None, save_flag, #2
                        image, filename) #3
```

1. Get a reference to the event loop.

2. The first argument is the executor to use; passing `None` selects the default `ThreadPoolExecutor` that is always available in `asyncio` event loop.

3. You can pass positional arguments to the function to run, but if you need to pass keyword arguments, then you need to resort to `functool.partial`, as described in [documentation](https://docs.python.org/3/library/asyncio-eventloop.html#asyncio.loop.run_in_executor).

## Writing asyncio Servers

We'll build slightly more interesting toys: server-side Unicode character search utilities, first using HTTP with `FastAPI`, then using plain TCP with `asyncio` only.

![search results for "mountain".](./img/2023-11-27-11-57-57.png)

### Meet the Inverted Index

An inverted index usually maps words to documents in which they occur. In the `mojifinder` examples, each “document” is one Unicode character. The `charindex.InvertedIndex` class indexes each word that appears in each character name in the Unicode database, and creates an inverted index stored in a `defaultdict` . For example, to index character U+0037—DIGIT SEVEN--the InvertedIndex initializer appends the character '7' to the entries under the keys 'DIGIT' and 'SEVEN'. 

![A demonstration using the entries for 'CAT' and 'FACE'](./img/2023-11-27-12-10-02.png)

### A FastAPI Web Service

`web_mojifinder.py` is a example  of the Python ASGI(Asynchronous Server Gateway Interface) Web frameworks. It's a super simple SPA(Single Page Application): after the initial HTML download, the UI is updated by client-side JavaScript communicating with the server.

`FastAPI` is designed to implement backends for SPA and mobile apps, which mostly consist of web API end points

## Asynchronous Iteration and Asynchronous Iterables

## async Beyond asyncio: Curio

## Type Hinting Asynchronous Objects

## How Async Works and How It doesn't

## Chapter Summary

## Further Reading